In [1]:
import sys
sys.path.insert(1, '../FeatureEngineering')

import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
embeddings = None

Spliteo para obtener vectores de train y test

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [3]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [4]:
processed, X_processed = prepare_for_cnn(train_df)

Percentage of words covered in the embeddings = 0.4937444933920705


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

In [29]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)])
    return model

In [30]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 1, 256)            473088    
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 1, 128)            164352    
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 740,417
Trainable params: 740,417
Non-trainable params: 0
_________________________________________________________________


In [31]:
history = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2)

Epoch 1/50
128/128 [==============================] - 4s 31ms/step - loss: 0.6782 - accuracy: 0.5809 - val_loss: 0.6752 - val_accuracy: 0.5265
Epoch 2/50
128/128 [==============================] - 2s 17ms/step - loss: 0.6276 - accuracy: 0.5841 - val_loss: 0.6469 - val_accuracy: 0.5422
Epoch 3/50
128/128 [==============================] - 2s 18ms/step - loss: 0.6040 - accuracy: 0.6179 - val_loss: 0.6368 - val_accuracy: 0.5735
Epoch 4/50
128/128 [==============================] - 2s 19ms/step - loss: 0.5825 - accuracy: 0.6603 - val_loss: 0.6142 - val_accuracy: 0.6794
Epoch 5/50
128/128 [==============================] - 2s 19ms/step - loss: 0.5589 - accuracy: 0.6973 - val_loss: 0.5953 - val_accuracy: 0.6804
Epoch 6/50
128/128 [==============================] - 2s 19ms/step - loss: 0.5352 - accuracy: 0.7294 - val_loss: 0.6059 - val_accuracy: 0.6353
Epoch 7/50
128/128 [==============================] - 2s 18ms/step - loss: 0.5095 - accuracy: 0.7456 - val_loss: 0.5597 - val_accuracy: 0.7343

In [32]:
prediction = model.predict_classes(X_test)

In [33]:
accuracy_score(y_test, prediction)

0.8093911659371269

In [34]:
f1_score(y_test, prediction)

0.7745882352941176